In [1]:
import math
from bprH import bprH, adv_index, save_model, load_model
import pandas as pd
from sklearn.model_selection import KFold
import numpy as np

In [2]:
def behavior_generator(num_user = 10, num_item=10):
    data = []
    for u in range(num_user):
        for i in range(num_item):
            p = np.random.uniform(0,1)
            if 0.7 <= p:
                data.append(['U' + str(u),'I' + str(i),'P'])
                data.append(['U' + str(u),'I' + str(i),'V'])
            elif 0.3 <= p:
                data.append(['U' + str(u),'I' + str(i),'V'])
    data = pd.DataFrame(data, columns=['UserID', 'ItemID', 'Action'])
    return data

In [3]:
data = behavior_generator(num_user=100, num_item=100)

In [4]:
# the best way is to split the data with a time manner - from sklearn.model_selection import TimeSeriesSplit
# but first let's do it in a simple way

original_item_list = sorted(set(data.ItemID))
original_user_list = sorted(set(data.UserID))

data_without_target = data[data.Action != 'P']
data_only_with_target = data[data.Action == 'P']

In [5]:
# split the data_only_with_target into 5 folds
kf = KFold(n_splits=5,shuffle=True, random_state=0)
kf.get_n_splits(X=data_only_with_target)

5

In [ ]:
cnt = 0
for train_index, test_index in kf.split(X=data_only_with_target):
    # build train and test data
    train_data = pd.concat([data_only_with_target.iloc[train_index], data_without_target])
    test_data = data_only_with_target.iloc[test_index]

    # start a BPRH model
    model = bprH(dim=10, omega=100, rho=1, lambda_u=0.01, lambda_v=0.01, lambda_b=0.01, gamma=0.005, random_state=20200704, num_iter=10000)

    model.fit(X=train_data, eval_X=test_data, original_item_list=original_item_list, original_user_list=original_user_list,
              saved_path='data/item-set-coselection-'+str(cnt)+'.pkl', coselection=True)
    cnt += 1


Calculate auxiliary-target correlation
Generate Itemset Coselection


ITER 37:   0%|          | 38/10000 [00:19<1:26:49,  1.91it/s, loss=-1.09, precision_10=0.0818, precision_5=0.0869, recall_10=0.139, recall_5=0.0694] 

In [2]:
# read sobazaar data
data = pd.read_csv('data/sb_unique_actions_20.csv')
# change column name
data.columns = ['ItemID', 'UserID', 'Action', 'Action_Date', 'Action_Time',
       'SessionID']

data = data.sample(frac=0.5) # TODO: test small data

In [3]:
# the best way is to split the data with a time manner - from sklearn.model_selection import TimeSeriesSplit
# but first let's do it in a simple way

original_item_list = sorted(set(data.ItemID))
original_user_list = sorted(set(data.UserID))

data_without_target = data[data.Action != 'P']
data_only_with_target = data[data.Action == 'P']

In [4]:
# split the data_only_with_target into 5 folds
kf = KFold(n_splits=2,shuffle=True, random_state=0)
kf.get_n_splits(X=data_only_with_target)

2

In [5]:
cnt = 0
for train_index, test_index in kf.split(X=data_only_with_target):
    # build train and test data
    train_data = pd.concat([data_only_with_target.iloc[train_index], data_without_target])
    test_data = data_only_with_target.iloc[test_index]

    # start a BPRH model
    model = bprH(dim=50, omega=1000, rho=1, lambda_u=0.5, lambda_v=0.5, lambda_b=0.5, gamma=0.01, random_state=20200704, num_iter=10000)

    model.fit(X=train_data, eval_X=test_data, original_item_list=original_item_list, original_user_list=original_user_list,
              saved_path='data/item-set-coselection-'+str(cnt)+'.pkl', coselection=False)
    cnt += 1

Calculate auxiliary-target correlation


ITER 9:   0%|          | 10/10000 [02:51<47:41:54, 17.19s/it, loss=-13.7, precision_10=0.000469, precision_5=0.000803, recall_10=0.00277, recall_5=0.0021]


KeyboardInterrupt: 

In [7]:
train_data = pd.concat([data_only_with_target.iloc[train_index], data_without_target])
test_data = data_only_with_target.iloc[test_index]

In [ ]:
# start a BPRH model
model = bprH(dim=50, omega=1000, rho=1, lambda_u=0.1, lambda_v=0.1, lambda_b=0.1, gamma=0.001, random_state=20200704)

model.fit(X=train_data, original_item_list=original_item_list, original_user_list=original_user_list, num_iter=10000, saved_path='data/item-set-coselection-'+str(cnt)+'.pkl')
cnt += 1

In [13]:
user_to_predict = sorted(set(test_data.UserID))

In [9]:
model = load_model(load_path='bprH_model.pkl')

In [14]:
model.scoring(ground_truth=test_data, K=5)

Build User's Purchased Item Dict & Rec List



  0%|          | 0/1271 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices